# Bring your Own Grid

Whilst Synference by default utilizes Synthesizer to create model libraries, it is also possible to bring your own pre-computed grid of models. This can be useful if you have a custom set of models that you wish to use for inference.

We provide a `LibraryCreator` class that allows you to create a grid from your own data and save it in the required HDF5 format.

In [ ]:
import os

import numpy as np
from astropy.table import Table

from synference import LibraryCreator

For this example we will build a grid from the SPHINX public data release, which can be found [here](https://github.com/HarleyKatz/SPHINX-20-data), and contains mock observations of 1380 galaxies from 10 different orientations at z=4.6 to 10 from the radiation-hydrodynamic cosmological simulation SPHINX.

In [ ]:
file = "https://raw.githubusercontent.com/HarleyKatz/SPHINX-20-data/refs/heads/main/data/all_basic_data.csv"


if not os.path.exists("all_basic_data.csv"):
    os.system(f"wget {file}")

sphinx = Table.read("all_basic_data.csv")

For the purposes of simplicity we will only use one direction (0) from the data release, but you could of course use all 10 directions if desired. 

We will create simple arrays containing the parameter and observation names we wish to use from the data release. In this case we will store redshift, stellar mass, stellar metallicity, mass-weighted age, SFR on 3 different timescales, and the dust E(B-V). For the observations we will use JWST NIRCam photometry.

We will also save some units information for the parameters, but this is optional.

In [ ]:
dir = 0  # Choose direction 0 for this example

parameter_columns = [
    "redshift",
    "stellar_mass",
    "stellar_metallicity",
    "mean_stellar_age_mass",
    "sfr_3",
    "sfr_10",
    "sfr_100",
    f"ebmv_dir_{dir}",
]
parameter_units = [
    "dimensionless",
    "log10(Msun)",
    "dimensionless",
    "Myr",
    "Msun/yr",
    "Msun/yr",
    "Msun/yr",
    "dimensionless",
]

feature_names = [
    f"F070W_dir_{dir}",
    f"F090W_dir_{dir}",
    f"F115W_dir_{dir}",
    f"F140M_dir_{dir}",
    f"F150W_dir_{dir}",
    f"F162M_dir_{dir}",
    f"F182M_dir_{dir}",
    f"F200W_dir_{dir}",
    f"F210M_dir_{dir}",
    f"F250M_dir_{dir}",
    f"F277W_dir_{dir}",
    f"F300M_dir_{dir}",
    f"F335M_dir_{dir}",
    f"F356W_dir_{dir}",
    f"F360M_dir_{dir}",
    f"F410M_dir_{dir}",
    f"F430M_dir_{dir}",
    f"F444W_dir_{dir}",
    f"F460M_dir_{dir}",
    f"F480M_dir_{dir}",
]

We now want to make a numpy array for the parameters and observations from the table. we want to make sure that the shape of these arrays matches the expected input for the `GridCreator` class, which is (n_models, n_parameters) and (n_models, n_observations) respectively. Therefore we need to transpose the arrays after converting them from the pandas dataframe.
```python

In [ ]:
parameters = sphinx[parameter_columns].to_pandas().to_numpy().T
features = sphinx[feature_names].to_pandas().to_numpy().T

If we wish we can also store some 'supplementary' parameters, which will not be inferred by default when we use the grid for inference, but can be accessed later if desired. This is useful for derived parameters or other quantities of interest. Here we will store the escape fraction, UV slope, and absolute UV magnitude.

In [ ]:
supplementary_columns = [f"fesc_dir_{dir}", f"beta_dir_{dir}_sn", f"MAB_1500_dir_{dir}"]
supplementary_units = ["dimensionless", "dimensionless", "AB"]

supplementary_data = sphinx[supplementary_columns].to_pandas().to_numpy().T

We will set nicer names for our features, and also define a feature transform function to convert the magnitudes to fluxes. The SPHINX data release provides magnitudes, but we will set input to be in nanoJanskys.

In [ ]:
override_feature_names = [f"JWST/NIRCam.{filter.split('_dir_')[0]}" for filter in feature_names]


def _feature_transform(features: np.ndarray) -> np.ndarray:
    # Convert AB mag to nJy
    flux = 10 ** (-0.4 * (features - 31.4))
    flux[features == 0] = 0  # Avoid division by zero
    flux[~np.isfinite(flux)] = 0  # Handle non-finite values
    return flux


features = _feature_transform(features)

Now we can create the grid using the `LibraryCreator` class. We will specify an output folder and set `overwrite=True` to overwrite any existing files.

In [ ]:
LibraryCreator(
    model_name="SPHINX_JWST",
    parameter_grid=parameters,
    observation_grid=features,
    observation_names=override_feature_names,
    observation_units="nJy",
    parameter_names=parameter_columns,
    parameter_units=parameter_units,
    supplementary_parameters=supplementary_data,
    supplementary_parameter_names=supplementary_columns,
    supplementary_parameter_units=supplementary_units,
    out_folder=".",
    overwrite=True,
)

Now let's quickly check that the grid was saved correctly by loading it back in using the SBI_Fitter class.


In [ ]:
from synference import SBI_Fitter

fitter = SBI_Fitter.init_from_hdf5(model_name="SPHINX_JWST", hdf5_path="./library_SPHINX_JWST.h5")

In [ ]:
print(fitter.raw_observation_names)
print(fitter.parameter_names)
print(fitter.parameter_units)

As we can see the grid has been loaded correctly with the expected parameter and observation names and units, and we could now proceed to use this grid for inference as normal.

This class provides a flexible way to bring your own model grids into Synference for inference.